In [1]:
import pandas as pd
import os
import tqdm
import cvxpy as cv
import numpy as np
cwd = os.getcwd()
cwd

'E:\\backtrader\\scripts'

In [2]:
data_path = r"E:\backtrader\CSMAR\日个股回报率文件\日个股回报率文件\daily_return.csv"

In [22]:
def clean(path, data_type='panel'):
    assert type in ['panel', 'ts'], 'type must be panel or ts'
    df = pd.read_csv(data_path)
    df.dropna(inplace=True)
    df['date'] = pd.to_datetime(df['date'])
    df['id'] = df['id'].astype('str').str.zfill(6)
    if type == 'panel':
        return df
    else:
        df = df.pivot(index='date', columns='id', values='daily_return')
        return df

df = clean(data_path, 'panel')
df

,id,date,daily_return_divident,daily_return,pre_close_price
0,000001,2018-11-01,-0.007333,-0.007333,10.910
1,000001,2018-11-02,0.024007,0.024007,10.830
2,000001,2018-11-05,-0.015329,-0.015329,11.090
3,000001,2018-11-06,-0.007326,-0.007326,10.920
4,000001,2018-11-07,-0.002768,-0.002768,10.840
...,...,...,...,...,...
5366784,900957,2023-10-25,0.017143,0.017143,0.350
5366785,900957,2023-10-26,0.022472,0.022472,0.356
5366786,900957,2023-10-27,0.016484,0.016484,0.364
5366787,900957,2023-10-30,0.062162,0.062162,0.370


In [26]:
df = clean(data_path, 'ts')
# fetch the date period from 2022-11-17 to the latest date
df_sample = df[df.index >= '2022-11-17'].dropna(axis=1)
df_sample

id,000001,000002,000005,000006,000008,000009,000010,000011,000012,000014,...,900941,900942,900943,900945,900946,900947,900948,900952,900953,900957
date,,,,,,,,,,,,,,,,,,,,,
2022-11-17,-0.010998,-0.003153,0.011236,0.004831,-0.004098,-0.012121,0.003077,0.010711,-0.020464,0.002215,...,0.007212,0.005376,0.053691,-0.008299,-0.009050,0.004049,-0.020901,0.003717,-0.027778,-0.014682
2022-11-18,-0.008554,-0.010120,-0.011111,-0.004808,-0.012346,0.035276,-0.009202,0.001927,-0.005571,0.008840,...,0.007160,-0.006684,-0.066242,-0.016736,-0.004566,0.004032,-0.021348,-0.007407,0.032967,0.008278
2022-11-21,-0.011217,-0.005112,0.022472,-0.009662,0.000000,0.000741,-0.009288,-0.015385,-0.001401,-0.009858,...,-0.016588,-0.012113,-0.009550,-0.029787,-0.009174,-0.004016,0.005453,-0.011194,-0.008511,0.003284
2022-11-22,0.031414,0.001927,0.005495,-0.004878,0.000000,-0.006662,0.012500,0.003906,-0.012623,-0.004425,...,0.031325,-0.002725,-0.089532,-0.008772,0.000000,0.008065,0.013559,0.011321,-0.012876,-0.006547
2022-11-23,0.000000,-0.007051,-0.027322,0.019608,0.016667,-0.009687,-0.009259,-0.010700,0.007102,0.016667,...,-0.023364,0.000000,-0.048411,-0.004425,0.101852,0.012000,0.009365,-0.007463,-0.010870,-0.001647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-25,-0.016114,0.003431,0.014184,0.014052,0.017241,0.000000,0.012690,0.007543,0.017408,-0.002660,...,0.034759,0.002894,-0.002188,0.000000,0.006135,0.005025,0.017126,0.020619,0.003289,0.017143
2023-10-26,0.002890,-0.011111,-0.020979,-0.011547,-0.004237,-0.002988,0.017544,-0.003209,0.001901,-0.010667,...,0.002584,0.007215,-0.028509,0.005650,-0.006098,0.000000,-0.008785,-0.005051,0.013115,0.022472
2023-10-27,0.003842,0.005186,0.014286,0.009346,0.004255,0.024975,-0.032020,0.006438,0.015180,0.010782,...,-0.002577,0.018625,0.011287,0.022472,0.018405,0.020000,0.005908,0.005076,0.016181,0.016484


In [31]:
cov_matrix = df_sample.cov().values
mu = df_sample.mean().values
expected_return_of_portfolio = 0.0005
rf = 0.035/365

In [30]:
cov_matrix
# check if the matrix is symmetric
def symmetric_check(matrix):
    return matrix[np.where(np.isclose(matrix, matrix.T, atol=1e-8) == False)]

array([], dtype=float64)

In [35]:
# 定义变量
weights = cv.Variable(len(mu))
#
# 定义目标函数
objective = cv.Minimize(cv.quad_form(weights, cov_matrix, assume_PSD=True))

# 定义约束条件
constraints = [
    cv.sum(weights) == 1,  # 权重之和为1
    weights.T @ mu == expected_return_of_portfolio,  # 期望收益率为给定值
    weights >= 0,  # 权重大于等于0
]

# 创建问题并求解
problem = cv.Problem(objective, constraints)
problem.solve()

# 输出结果
print("最优权重:", weights.value)
print("最小方差:", problem.value)


最优权重: [-9.97690295e-09 -1.69864057e-08  3.55224491e-08 ...  2.49730840e-08
 -5.39012160e-08 -7.88673078e-08]
最小方差: 9.79388321138593e-06


In [37]:
threshold = 0.0001
weights.value[np.abs(weights.value) < threshold] = 0
weights.value[weights.value > 0]

array([0.00111924, 0.00872369, 0.00277365, 0.0035054 , 0.00699962,
       0.00206251, 0.01268154, 0.02834071, 0.00221352, 0.01479044,
       0.00160027, 0.00312037, 0.00642464, 0.00484507, 0.01200458,
       0.00358681, 0.01103271, 0.00326854, 0.00658707, 0.00538134,
       0.01392531, 0.0146464 , 0.03534426, 0.01603583, 0.01461601,
       0.02410976, 0.02391691, 0.01195348, 0.01334307, 0.00562642,
       0.01620639, 0.00126995, 0.01194302, 0.01971942, 0.00695478,
       0.00316571, 0.00943303, 0.00299895, 0.00791339, 0.0060739 ,
       0.00498692, 0.00019078, 0.01830271, 0.00139415, 0.00124439,
       0.01318722, 0.00044244, 0.00376824, 0.00399697, 0.00074333,
       0.00415893, 0.01112944, 0.01346342, 0.00724324, 0.00482032,
       0.00463368, 0.00176406, 0.0109019 , 0.01334814, 0.01178146,
       0.01173111, 0.00056431, 0.00391974, 0.00254013, 0.00341808,
       0.00169179, 0.0115474 , 0.01013958, 0.00606674, 0.00331527,
       0.01948182, 0.01190019, 0.0001428 , 0.00689566, 0.00043

In [40]:
portfolio_return = weights.value @ mu
portfolio_risk = problem.value ** 0.5
# 输出组合收益率，组合风险和夏普比率
print("组合收益率:", portfolio_return)
print("组合风险:", portfolio_risk)
print("夏普比率:", (portfolio_return - rf) / portfolio_risk)

组合收益率: 0.0005000376528627148
组合风险: 0.0031295180477808287
夏普比率: 0.12914040939639104
